In [54]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np

def test_filter(site1, input_raw1, site2, input_raw2):

    for raw_one1 in input_raw1:
        for raw_one2 in input_raw2:

            if site1 == 'NTIS' :
                #NTIS 
                print('ntis')
                if raw_one1.get('rsc') == None:
                    coauthor1 = 0
                else:
                    coauthor1 = raw_one1.get('rsc').split(";")

                if raw_one1.get('koKeyword') == None:
                    keyword1 = ""
                else:
                    keyword1 = raw_one1.get('koKeyword')

            else:
                journal1 = raw_one1.get('journal')
                issInst1 = raw_one1.get('issue_inst')
                issYear1 = int(raw_one1.get('issue_year')[:4])
                coauthor1 = raw_one1.get('author').split(";")[1:-1]
                keyword1 = raw_one1.get('paper_keyword')

            if site2 == 'NTIS' :
                #NTIS
                print('ntis')
                if raw_one2.get('rsc') == None:
                    coauthor2 = 0
                else:
                    coauthor2 = raw_one2.get('rsc').split(";")

                if raw_one2.get('koKeyword') == None:
                    keyword2 = ""
                else:
                    keyword2 = raw_one2.get('koKeyword')

            else:
                journal2 = raw_one2.get('journal')
                issInst2 = raw_one2.get('issue_inst')
                issYear2 = int(raw_one2.get('issue_year')[:4])
                coauthor2 = raw_one2.get('author').split(";")[1:-1]
                keyword2 = raw_one2.get('paper_keyword')

            weight = 0
            
            if journal1 == journal2 and issInst1 == issInst2:
                weight += 1
            else:
                weight -= 1
            
            Yop = -(2*((issYear1-issYear2)/10)-1)
            co_authorship = 1 - np.exp(-len([i for i in coauthor1 if i in coauthor2]))
            keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

            weight += Yop + co_authorship + keyword

    return weight

In [55]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

site1 = 'Scienceon'
site2 = 'Scienceon'
input_paper1 = ObjectId('61939443e9b04a9d64abe055')
input_paper2 = ObjectId('61939447e9b04a9d64abe06a')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find({ '_id' : input_paper1 })
input_raw2 = site_raw2.find({ '_id' : input_paper2 })

print(test_filter(site1, input_raw1, site2, input_raw2))

{'_id': ObjectId('61939443e9b04a9d64abe055'), 'qryKeyword': ['빅데이터', '소셜네트워크'], 'qryTime': '202111162021', 'keyId': 588, 'loop': 4, 'issue_inst': '한국콘텐츠학회', 'journal': '한국콘텐츠학회논문지 = The Journal of the Korea Contents Association', 'issue_year': '2019', 'id': 'JAKO201909469054948', 'title': '온라인 쇼핑몰에서 상품 신뢰도를 고려한 개인화 추천', 'english_title': 'Personalized Recommendation Considering Item Confidence in E-Commerce', 'abstract': '온라인 쇼핑몰의 대중화로 인해 소비자는 폭 넓은 소비의 기회를 제공받고 있다. 소비자들은 온라인 쇼핑몰에서 제공되는 실제 상품을 구매한 사용자의 리뷰, 상품의 자세한 정보와 같은 정보를 활용하여 상품의 구매 여부를 결정한다. 방대한 정보가 신뢰할 수 있는 정보인지는 소비자들이 스스로 판단해야만하기 때문에 객관적이며 신뢰할 수 있는 정보의 제공이 필요하다. 본 논문에서는 신뢰할 수 있는 상품의 추천을 위해서 상품의 신뢰도를 고려한 개인화된 추천 기법을 제안한다. 제안하는 추천 기법은 개인화 추천을 위해서 사용자의 다양한 행위를 기반으로 사용자의 선호도를 판별한다. 또한, 최신의 성향을 반영하기 위해서 시간 가중치를 고려한 사용자 선호도 계산 방법을 제안한다. 마지막으로 사용자가 사용하지 않았던 상품에 대한 선호 점수를 예측하고 예측 점수가 높은 상품 중 신뢰도가 높은 상품들을 추천한다.', 'english_abstract': '', 'citation': 0, 'author': '최도진;박재열;박수빈;임종태;송재오;복경수;유재수;', 'author_inst': '충북대학교 정보통신공학과;충북대학교 정보통신공학과;충북대